In [1]:
# Importa o módulo principal do Keras, uma API de alto nível para criação de redes neurais.
# Em versões modernas, Keras está integrado ao TensorFlow.
import keras

# Importa o TensorFlow, uma biblioteca amplamente usada para aprendizado profundo e computação numérica.
# Ele serve como backend para Keras, fornecendo as operações de baixo nível necessárias.
import tensorflow as tf


In [2]:
# Importa o conjunto de dados MNIST, que contém imagens de dígitos escritos à mão (0-9).
# Este conjunto de dados é amplamente utilizado em tarefas de classificação de imagens.
from tensorflow.keras.datasets import mnist

# Importa a classe `Sequential` para criar modelos de rede neural sequenciais (camada a camada).
from tensorflow.keras.models import Sequential

# Importa diversas camadas utilizadas em redes neurais:
# - InputLayer: Define a camada de entrada.
# - Dense: Camada totalmente conectada.
# - Flatten: Transforma entradas multidimensionais em vetores.
# - Conv2D: Camada de convolução para processamento de imagens.
# - MaxPooling2D: Camada de pooling para redução dimensional.
from tensorflow.keras.layers import InputLayer, Dense, Flatten, Conv2D, MaxPooling2D

# Importa a classe `ImageDataGenerator`, usada para realizar aumento de dados (data augmentation).
# O aumento de dados gera versões transformadas (rotacionadas, ampliadas, etc.) das imagens,
# ajudando a melhorar a generalização do modelo.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Importa o módulo `utils` do Keras, com alias `np_utils`, que fornece ferramentas úteis,
# como a função `to_categorical` para converter rótulos em formato one-hot.
from tensorflow.keras import utils as np_utils


In [3]:
# Carrega o conjunto de dados MNIST.
# `mnist.load_data()` retorna duas tuplas:
# - `(X_treinamento, y_treinamento)`: Dados de treinamento (imagens e rótulos correspondentes).
# - `(X_teste, y_teste)`: Dados de teste (imagens e rótulos correspondentes).
# As imagens são representadas como arrays NumPy com formato (28, 28), contendo valores de intensidade de pixels (0 a 255).
(X_treinamento, y_treinamento), (X_teste, y_teste) = mnist.load_data()


In [4]:
# Redimensiona os dados de treinamento para incluir um canal adicional (28x28x1),
# necessário para processamento em redes convolucionais que esperam entradas com 3 ou 4 dimensões.
# `X_treinamento.shape[0]` é o número total de exemplos no conjunto de treinamento.
X_treinamento = X_treinamento.reshape(X_treinamento.shape[0], 28, 28, 1)

# Redimensiona os dados de teste para o mesmo formato (28x28x1).
X_teste = X_teste.reshape(X_teste.shape[0], 28, 28, 1)

# Converte os valores dos pixels das imagens de inteiros (`uint8`) para ponto flutuante (`float32`).
# Isso é necessário para normalizar os valores.
X_treinamento = X_treinamento.astype('float32')
X_teste = X_teste.astype('float32')

# Normaliza os valores dos pixels para o intervalo [0, 1].
# Isso melhora a estabilidade numérica durante o treinamento.
X_treinamento /= 255
X_teste /= 255

# Converte os rótulos de inteiros (0-9) para o formato one-hot encoding.
# Em vez de um único valor para cada rótulo, cria-se um vetor binário de comprimento 10.
# Exemplo: O rótulo 3 é transformado em [0, 0, 0, 1, 0, 0, 0, 0, 0, 0].
y_treinamento = np_utils.to_categorical(y_treinamento, 10)
y_teste = np_utils.to_categorical(y_teste, 10)


In [6]:
# Cria um modelo sequencial, onde as camadas são empilhadas uma após a outra.
classificador = Sequential()

# Adiciona uma camada de entrada para processar imagens no formato 28x28x1 (grayscale).
classificador.add(InputLayer(shape=(28, 28, 1)))

# Adiciona uma camada convolucional 2D com:
# - 32 filtros, cada um de tamanho 3x3.
# - Função de ativação 'relu', que introduz não-linearidade.
# Essa camada aprende características locais, como bordas e texturas.
classificador.add(Conv2D(32, (3, 3), activation='relu'))

# Adiciona uma camada de max pooling com uma janela de 2x2.
# Essa camada reduz a dimensionalidade das representações aprendidas,
# ajudando a reduzir o custo computacional e evitar overfitting.
classificador.add(MaxPooling2D(pool_size=(2, 2)))

# Achata a saída das camadas anteriores, transformando a matriz em um vetor unidimensional.
# Isso é necessário antes de conectar as camadas densas.
classificador.add(Flatten())

# Adiciona uma camada totalmente conectada (densa) com 128 neurônios e ativação 'relu'.
# Essa camada aprende representações mais abstratas e complexas.
classificador.add(Dense(units=128, activation='relu'))

# Adiciona a camada de saída com 10 neurônios (um para cada classe) e ativação 'softmax'.
# A função 'softmax' converte os valores de saída em probabilidades, somando 1.
classificador.add(Dense(units=10, activation='softmax'))

# Compila o modelo especificando:
# - `loss='categorical_crossentropy'`: Função de perda para classificação multiclasse com rótulos one-hot.
# - `optimizer='adam'`: Algoritmo de otimização que ajusta os pesos durante o treinamento.
# - `metrics=['accuracy']`: Métrica de avaliação que monitora a precisão durante o treinamento e validação.
classificador.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [7]:
# Cria um objeto `ImageDataGenerator` para realizar aumento de dados (data augmentation).
# O aumento de dados gera versões transformadas das imagens, ajudando a melhorar a generalização do modelo,
# especialmente quando o conjunto de dados é pequeno ou quando queremos evitar overfitting.

gerador_treinamento = ImageDataGenerator(
    rotation_range=7,         # Gira as imagens aleatoriamente em até 7 graus.
    horizontal_flip=True,     # Permite a inversão horizontal aleatória das imagens.
    shear_range=0.2,          # Aplica cisalhamento aleatório em até 20%.
    height_shift_range=0.07,  # Desloca as imagens verticalmente em até 7% da altura.
    zoom_range=0.2            # Aplica zoom aleatório em até 20% das imagens.
)


In [12]:
# Cria um objeto `ImageDataGenerator` para o conjunto de dados de teste.
# No caso do conjunto de teste, normalmente não aplicamos aumento de dados, 
# pois queremos avaliar o modelo em dados não modificados.
# Portanto, o `ImageDataGenerator` é instanciado sem nenhuma transformação.
gerador_teste = ImageDataGenerator()


In [13]:
# Cria um gerador de dados para o conjunto de treinamento utilizando o `ImageDataGenerator` definido anteriormente.
# O método `flow` gera batches de imagens e rótulos, realizando o aumento de dados em tempo real.
# - `X_treinamento`: Dados de entrada (imagens) para o treinamento.
# - `y_treinamento`: Rótulos correspondentes às imagens.
# - `batch_size=128`: Número de amostras por batch durante o treinamento.
# Esse gerador será usado para alimentar o modelo durante o treinamento, aplicando as transformações definidas no `gerador_treinamento`.
base_treinamento = gerador_treinamento.flow(X_treinamento, y_treinamento, batch_size=128)


In [10]:
# Cria um gerador de dados para o conjunto de teste utilizando o `ImageDataGenerator` definido anteriormente.
# O método `flow` gera batches de imagens e rótulos para o conjunto de teste, mas sem aplicar aumentos de dados,
# já que o conjunto de teste não deve ser alterado para avaliação do modelo.
# - `X_teste`: Dados de entrada (imagens) para o teste.
# - `y_teste`: Rótulos correspondentes às imagens.
# - `batch_size=128`: Número de amostras por batch durante a avaliação.
# Esse gerador será usado para alimentar o modelo durante a avaliação ou validação.
base_teste = gerador_teste.flow(X_teste, y_teste, batch_size=128)


In [14]:
# Treina o modelo utilizando o gerador de dados para o conjunto de treinamento.
# O método `fit` recebe os geradores de dados em vez de arrays diretos, permitindo o uso do aumento de dados em tempo real.
# - `base_treinamento`: Gerador que fornece os dados de treinamento com aumento de dados.
# - `epochs=5`: Número de épocas (iterações completas sobre o conjunto de treinamento).
# - `validation_data=base_teste`: Conjunto de dados para validação, neste caso o gerador para o conjunto de teste.
# O modelo será treinado por 5 épocas e, a cada época, a precisão será calculada no conjunto de teste usando `base_teste`.
classificador.fit(base_treinamento, epochs=5, validation_data=base_teste)


Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 18s 39ms/step - accuracy: 0.9639 - loss: 0.1157 - val_accuracy: 0.9746 - val_loss: 0.0800
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 18s 38ms/step - accuracy: 0.9657 - loss: 0.1100 - val_accuracy: 0.9761 - val_loss: 0.0732
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 17s 36ms/step - accuracy: 0.9692 - loss: 0.0988 - val_accuracy: 0.9686 - val_loss: 0.0981
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 17s 36ms/step - accuracy: 0.9698 - loss: 0.0972 - val_accuracy: 0.9800 - val_loss: 0.0623
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 17s 35ms/step - accuracy: 0.9714 - loss: 0.0894 - val_accuracy: 0.9748 - val_loss: 0.0754
